### **PIPELINE ДЛЯ PRIMARY ДАННЫХ (TEST.CSV)**  
---

Данный ноутбук обрабатывает test.csv и выгружает целевую таблицу inference_primary с true_label и prediction_score в Платформенную БД.

In [ ]:
pip install psycopg2-binary=="2.9.9" boto3=="1.35.0" pyarrow fastparquet

In [ ]:
import pandas as pd
import numpy as np
import boto3
import tempfile
import os
from sqlalchemy import create_engine
from sklearn.preprocessing import LabelEncoder

In [ ]:
def setup_s3_client():
    project_s3 = boto3.resource(
        "s3",
        endpoint_url=os.environ.get("FEAST_S3_ENDPOINT_URL"),
        aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID"), 
        aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY")
    )
    
    bucket_name = os.environ.get("S3_BUCKET")
    print(f"Бакет: {bucket_name}")
    
    return project_s3, bucket_name

In [ ]:
def create_db_connection(password):
    DATABASE_USER = "vectoradmin"
    DATABASE_PASSWORD = password
    DATABASE_HOST = os.environ.get("PGVECTOR_HOST_NAME")
    DATABASE_PORT = 5432
    DATABASE_DBNAME = os.environ.get("PGVECTOR_DB_NAME")
    
    connection_string = f"postgresql://{DATABASE_USER}:{DATABASE_PASSWORD}@{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_DBNAME}"
    
    engine = create_engine(connection_string)
    
    # Проверка подключения
    try:
        with engine.connect() as conn:
            print("Подключение к БД установлено")
        return engine
    except Exception as e:
        print(f"Ошибка подключения к БД: {e}")
        raise

In [ ]:
def upload_df_to_s3_parquet(project_s3, df, bucket_name, s3_key):
    """
    Загружает DataFrame в S3 как Parquet файл
    
    Args:
        project_s3: S3 клиент
        df: DataFrame для сохранения
        bucket_name: Имя S3 бакета
        s3_key: Ключ (путь) для сохранения файла в S3
    """
    with tempfile.NamedTemporaryFile(suffix='.parquet', delete=False) as tmp_file:
        df.to_parquet(tmp_file.name, index=False)
        
        project_s3.Bucket(bucket_name).upload_file(tmp_file.name, s3_key)

        os.unlink(tmp_file.name)

In [ ]:
def find_and_read_first_parquet(project_s3, bucket_name, folder_path=""):
    """
    Находит и читает parquet файл в указанной папке S3
    (по дате последнего изменения)
    
    Args:
        project_s3: S3 клиент
        bucket_name: Имя S3 бакета
        folder_path: Путь к папке в S3
        
    Returns:
        DataFrame с данными из самого нового parquet файла
    """
    if folder_path and not folder_path.endswith('/'):
        folder_path += '/'
    
    # Получаем список объектов
    objects = list(project_s3.Bucket(bucket_name).objects.filter(Prefix=folder_path))
    
    # Фильтруем только parquet файлы
    parquet_files = [obj for obj in objects if obj.key.endswith('.parquet')]
    
    if not parquet_files:
        raise FileNotFoundError(f"arquet файлы не найдены в s3://{bucket_name}/{folder_path}")
    
    parquet_files_sorted = sorted(
        parquet_files, 
        key=lambda x: x.last_modified, 
        reverse=True
    )
    
    latest_parquet = parquet_files_sorted[0]
    s3_key = latest_parquet.key
    
    print(f"Найден файл: {s3_key}")
    print(f"Дата изменения: {latest_parquet.last_modified}")
    # Читаем файл
    with tempfile.NamedTemporaryFile(suffix='.parquet', delete=False) as tmp_file:
        project_s3.Bucket(bucket_name).download_file(s3_key, tmp_file.name)
        df = pd.read_parquet(tmp_file.name)
        os.unlink(tmp_file.name)
    
    print(f"Загружен файл из S3: {s3_key}")
    return df

In [ ]:
def process_test_data(project_s3, test_csv_path, bucket_name):
    """
    Обрабатывает test.csv файл для создания primary данных
    
    Args:
        project_s3: S3 клиент
        test_csv_path: Путь к test.csv файлу
        bucket_name: Имя S3 бакета
    """
    df = pd.read_csv(test_csv_path)
    
    print("Исходные данные test:")
    print(f"Размер: {df.shape}")
    print(f"Колонки: {list(df.columns)}")
    
    df['app_date'] = pd.to_datetime(df['app_date'], format='%d%b%Y')
    metadata_cols = df[['client_id', 'app_date']].copy()
    
    # Применяем преобразования
    
    df['age'] = np.log(df['age'] + 1)
    df['decline_app_cnt'] = np.log(df['decline_app_cnt'] + 1)
    df['bki_request_cnt'] = np.log(df['bki_request_cnt'] + 1)
    df['income'] = np.log(df['income'] + 1)
    
    df['education'] = df['education'].fillna('SCH')
    
    start = df.app_date.min()
    df['days'] = (df.app_date - start).dt.days.astype('int')
    
    bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']
    label_encoder = LabelEncoder()
    for column in bin_cols:
        df[column] = label_encoder.fit_transform(df[column])
    
    cat_cols = ['education', 'region_rating', 'home_address', 'work_address', 'sna', 'first_time']
    df = pd.get_dummies(df, prefix=cat_cols, columns=cat_cols, dtype=int)
    
    # Удаляем мета-колонки для батч-версии
    columns_to_drop = ['app_date', 'client_id']
    df_for_batch = df.drop(columns_to_drop, axis=1, errors='ignore')
    
    print("После преобразований:")
    print(f"Для batch инференса: {df_for_batch.shape}")
    print(f"Колонки для batch: {list(df_for_batch.columns)}")
    
    upload_df_to_s3_parquet(project_s3, df_for_batch, bucket_name, 'primary_features.parquet')
    
    print(f"Test данные обработаны. Primary features сохранены в S3")
    
    return df_for_batch, metadata_cols


In [ ]:
def load_and_combine_primary(predictions_s3_key, original_metadata):
    """
    Загружает DataFrame с предсказаниями в PostgreSQL базу данных с отклонением ±5% от prediction_score
    
    Args:
        project_s3: S3 клиент
        bucket_name: Имя S3 бакета
        df: DataFrame для загрузки
        table_name: Имя таблицы в БД
    """
    df_batch = find_and_read_first_parquet(project_s3, bucket_name, predictions_s3_key)
    
    df_combined = pd.concat([
        original_metadata.reset_index(drop=True), 
        df_batch.reset_index(drop=True)
    ], axis=1)
    
    # Генерируем true_label с отклонением ±40%
    np.random.seed(42)
    
    if 'prediction_score' in df_combined.columns:
        deviation = 0.4
        base_proba = df_combined['prediction_score'].values
        
        noise = np.random.uniform(-deviation, deviation, size=len(df_combined))
        noisy_proba = np.clip(base_proba + noise, 0, 1)
        
        df_combined['true_label'] = (np.random.random(len(df_combined)) < noisy_proba).astype(int)
    else:
        df_combined['app_date'] = pd.to_datetime(df_combined['app_date'], errors='coerce')
    
    
    df_combined['app_date'] = pd.to_datetime(df_combined['app_date'])
    df_combined['app_date'] = df_combined['app_date'].apply(lambda x: x.replace(year=2024))
    
    print(f"Данные объединены: {df_combined.shape}")
    print(f"   Дефолтов: {df_combined['true_label'].sum()}/{len(df_combined)} ({df_combined['true_label'].mean():.1%})")
    
    return df_combined

In [ ]:
def load_and_combine_primary_test(predictions_s3_key, original_metadata, deviation_percent=15.0):
    """
    Загружает DataFrame с предсказаниями и генерирует true_label так,
    чтобы метрики были с отклонением -deviation_percent% от baseline-значений.
    """
    
    df_batch = find_and_read_first_parquet(project_s3, bucket_name, predictions_s3_key)
    df_combined = pd.concat([
        original_metadata.reset_index(drop=True), 
        df_batch.reset_index(drop=True)
    ], axis=1)
    
    # Baseline метрики
    baseline_metrics = {
        'accuracy': 0.670528,
        'precision': 0.226289,
        'recall': 0.686918,
        'f1': 0.340431
    }
    
    np.random.seed(42)
    
    if 'prediction_score' in df_combined.columns:
        base_proba = df_combined['prediction_score'].values
        y_pred = (base_proba > 0.5).astype(int)
        n = len(y_pred)
        
        true_label = y_pred.copy()
        
        # 2. Ухудшаем accuracy на deviation_percent%
        target_acc = baseline_metrics['accuracy'] * (1 - deviation_percent/100)
        current_acc = np.mean(true_label == y_pred)
        
        # Сколько нужно изменить предсказаний для достижения target_acc
        n_to_change = int(abs(current_acc - target_acc) * n)
        
        if current_acc > target_acc:  # Нужно ухудшить accuracy
            # Находим индексы где предсказание правильное
            correct_idx = np.where(true_label == y_pred)[0]
            # Случайно выбираем часть правильных и меняем на противоположные
            change_idx = np.random.choice(correct_idx, size=n_to_change, replace=False)
            true_label[change_idx] = 1 - true_label[change_idx]
        
        # 3. Ухудшаем precision на deviation_percent%
        target_prec = baseline_metrics['precision'] * (1 - deviation_percent/100)
        pred_pos_idx = np.where(y_pred == 1)[0]
        
        if len(pred_pos_idx) > 0:
            # Вычисляем сколько должно быть TP
            target_TP = int(target_prec * len(pred_pos_idx))
            current_TP = np.sum((y_pred[pred_pos_idx] == 1) & (true_label[pred_pos_idx] == 1))
            
            if current_TP > target_TP:
                # Находим TP которые нужно превратить в FP
                tp_idx = pred_pos_idx[(y_pred[pred_pos_idx] == 1) & (true_label[pred_pos_idx] == 1)]
                n_to_change = current_TP - target_TP
                change_idx = np.random.choice(tp_idx, size=min(n_to_change, len(tp_idx)), replace=False)
                true_label[change_idx] = 0
        
        df_combined['true_label'] = true_label
    else:
        df_combined['true_label'] = np.random.choice([0, 1], size=len(df_combined))
    
    # Проверяем результат
    y_true = df_combined['true_label'].values
    y_pred = (df_combined['prediction_score'].values > 0.5).astype(int) if 'prediction_score' in df_combined.columns else None
    
    if y_pred is not None:
        from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
        
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)
    
    return df_combined

In [ ]:
def load_predictions_to_db(project_s3, bucket_name, df, table_name, vectoradmin_password):
    """
    Загружает DataFrame с предсказаниями в PostgreSQL базу данных
    
    Args:
        project_s3: S3 клиент
        bucket_name: Имя S3 бакета
        df: DataFrame для загрузки
        table_name: Имя таблицы в БД
    """
    db_connection = create_db_connection(vectoradmin_password)
    
    df.to_sql(table_name, db_connection, if_exists='replace', index=False)
    
    print(f"Создана/обновлена таблица '{table_name}' с {len(df)} записями")
    
    # with db_connection.connect() as conn:
    #     result = conn.execute(f"SELECT COUNT(*) FROM {table_name}")
    #     count = result.scalar()
    #     print(f"Проверка: в таблице {table_name} {count} записей")


In [ ]:
def add_degraded_window(predictions_s3_key, original_metadata, vectoradmin_password, degradation=0.10, sample_size=1000):
    """
    Создает деградированные данные и добавляет их в таблицу inference_primary через APPEND
    """
    # Загружаем данные
    df_batch = find_and_read_first_parquet(project_s3, bucket_name, predictions_s3_key)
    df_combined = pd.concat([original_metadata.reset_index(drop=True), df_batch.reset_index(drop=True)], axis=1)
    
    # Baseline метрики
    baseline = {
        'accuracy': 0.670528,
        'precision': 0.226289, 
        'recall': 0.686918,
        'f1': 0.340431
    }
    
    # Целевые метрики
    target = {k: v * (1 - degradation) for k, v in baseline.items()}
    
    np.random.seed(42)
    
    if 'prediction_score' in df_combined.columns:
        proba = df_combined['prediction_score'].values
        y_pred = (proba > 0.5).astype(int)
        n = len(y_pred)
        
        # Начинаем с правильных предсказаний
        y_true = y_pred.copy()
        
        # Добавляем ошибки для достижения target accuracy
        target_correct = int(n * target['accuracy'])
        errors_needed = n - target_correct
        
        if errors_needed > 0:
            error_indices = np.random.choice(n, size=errors_needed, replace=False)
            for idx in error_indices:
                y_true[idx] = 1 - y_true[idx]
        
        df_combined['true_label'] = y_true
        
        # Проверяем метрики
        from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
        
        achieved = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred, zero_division=0),
            'recall': recall_score(y_true, y_pred, zero_division=0),
            'f1': f1_score(y_true, y_pred, zero_division=0)
        }
        
#         print(f"Target metrics ({degradation*100:.0f}% degradation):")
#         for metric, value in target.items():
#             print(f"  {metric}: {value:.4f}")
        
#         print("\nAchieved metrics:")
#         for metric, value in achieved.items():
#             print(f"  {metric}: {value:.4f}")
        
#         print("\nDeviation from baseline (%):")
#         for metric in baseline:
#             deviation = ((achieved[metric] - baseline[metric]) / baseline[metric]) * 100
#             print(f"  {metric}: {deviation:.1f}%")
            
#             if abs(deviation) > 15:
#                 print(f"THRESHOLD TRIGGERED! > 15%")
    
    else:
        df_combined['true_label'] = np.random.choice([0, 1], size=len(df_combined), p=[0.85, 0.15])
    
    db_connection = create_db_connection(vectoradmin_password)
    
    df_combined['app_date'] = pd.to_datetime(df_combined['app_date'])
    df_combined['app_date'] = df_combined['app_date'].apply(lambda x: x.replace(year=2025, month=11, day=30))
    
    # Добавляем в БД
    try:
        df_combined.to_sql('inference_primary', db_connection, if_exists='append', index=False)
        print("Данные успешно добавлены!")
    except Exception as e:
        print(f"Ошибка: {e}")
    finally:
        db_connection.dispose()
    
    return df_combined

### **MAIN PIPELINE**
---

Основной пайплайн для обработки **primary** данных<br>
Выполнять последовательно все шаги:<br>
1. Настройка подключений<br>
2. Обработка test.csv<br>
3. **(Ручной шаг) Запуск батч-сервиса на primary_features.parquet**<br>
4. Загрузка и обработка предсказаний с генерацией synthetic true_label<br>
6. **Перед следующим шагом необходимо посмотреть пароль для бд пгвектора и ввести параметр**<br>
5. Загрузка в БД<br>

### **Настройка подключений**

In [ ]:
project_s3, bucket_name = setup_s3_client()

### **Обработка test данных**

In [ ]:
primary_data, metadata_cols = process_test_data(
        project_s3, 
        "data/test.csv", 
        bucket_name
    )
print("РУЧНОЙ ШАГ: ЗАПУСТИТЕ БАТЧ-СЕРВИС")
print("Используйте primary_features.parquet из S3 для инференса")
print(f"INPUT_DATA: s3a://{bucket_name}/primary_features.parquet")
print(f"OUTPUT_DIRECTORY: s3a://{bucket_name}/inference_primary")

### **Загрузка primary предсказаний**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

df_combined = load_and_combine_primary_test("inference_primary/", metadata_cols, 20)

In [ ]:
df_combined

### **Загрузка в базу данных**

In [ ]:
 load_predictions_to_db(project_s3, bucket_name, df_combined, "inference_primary", "vectoradmin_password")

### **Добавление новых данных в таблицу**
**Необходимо посмотреть пароль для бд пгвектора и ввести параметр**

In [ ]:
new_window = add_degraded_window(
    predictions_s3_key="inference_primary/",
    original_metadata=metadata_cols,
    degradation=0.1,
    vectoradmin_password="vectoradmin_password"
)